# 마이크로 컨트롤러 사용을 위한 간단한 음성 인식 모델 훈련

지금부터 살펴볼 코드는 [간단한 음성 인식](https://www.tensorflow.org/tutorials/sequences/audio_recognition)을 위한 20kb 정도의 마이크로 컨트롤러용 [TensorFlow Lite 모델](https://tensorflow.org/lite/microcontrollers/overview)을 훈련하는 방법을 살펴볼 것이다. 이 모델은 [micro_speech](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/lite/micro/examples/micro_speech) 예제 애플리케이션에서 사용된 것과 같다.

이 모델은 [Google Colaboratory](https://colab.research.google.com) 환경에서도 사용할 수 있다. 

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/lite/micro/examples/micro_speech/train_speech_model.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/micro/examples/micro_speech/train_speech_model.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>


이 노트북의 코드는 Python 스크립트를 실행하여 훈련을 진행한 뒤, 모델을 고정시킨다. 그리고 Tensorflow Lite 모델 변환기를 통해 마이크로컨트롤러용 모델을 생성한다. 

**GPU 가속을 사용하면 훈련 속도가 훨씬 빨라진다.** 훈련을 하기에 앞서, 런타임 유형을 확인해보자. **Runtime -> Change runtime type**을 클릭하면 런타임을 **GPU** 환경으로 바꿀 수 있다. 훈련에는 GPU환경을 기준으로 18,000  이터레이션의 경우 1시간반에서 2시간 정도 걸린다.

## 훈련 설정하기

다음 `os.environ` 라인에서 학습할 단어, 훈련단계 및 학습률을 설정할 수 있다. 기본값은 micro_speech 예제에서 사용 된 것과 동일하다. 셀을 실행하여 설정을 진행하자:

In [ ]:
import os

# A comma-delimited list of the words you want to train for.
# The options are: yes,no,up,down,left,right,on,off,stop,go
# All other words will be used to train an "unknown" category.
os.environ["WANTED_WORDS"] = "yes,no"

# The number of steps and learning rates can be specified as comma-separated
# lists to define the rate at each stage. For example,
# TRAINING_STEPS=15000,3000 and LEARNING_RATE=0.001,0.0001
# will run 18,000 training loops in total, with a rate of 0.001 for the first
# 15,000, and 0.0001 for the final 3,000.
os.environ["TRAINING_STEPS"]="15000,3000"
os.environ["LEARNING_RATE"]="0.001,0.0001"

# Calculate the total number of steps, which is used to identify the checkpoint
# file name.
total_steps = sum(map(lambda string: int(string),
                  os.environ["TRAINING_STEPS"].split(",")))
os.environ["TOTAL_STEPS"] = str(total_steps)

# Print the configuration to confirm it
!echo "Training these words: ${WANTED_WORDS}"
!echo "Training steps in each stage: ${TRAINING_STEPS}"
!echo "Learning rate in each stage: ${LEARNING_RATE}"
!echo "Total number of training steps: ${TOTAL_STEPS}"


## 종속성 설치

다음으로, Tensorflow의 GPU 빌드 버전을 설치할 것이다. GPU를 지원하는 빌드 버전을 설치해야 GPU가속을 훈련시 사용할 수 있다. 

In [ ]:
# Replace Colab's default TensorFlow install with a more recent
# build that contains the operations that are needed for training
!pip uninstall -y tensorflow tensorflow_estimator tensorboard
!pip install -q tf-estimator-nightly==1.14.0.dev2019072901 tf-nightly-gpu==1.15.0.dev20190729

git 명령어를 활용하여 모델을 훈련시키고, 훈련 모델을 고정시키는 스크립트가 포함된 TensorFlow 레포지토리를 복제하도록 하자. 

In [ ]:
# Clone the repository from GitHub
!git clone -q https://github.com/tensorflow/tensorflow
# Check out a commit that has been tested to work
# with the build of TensorFlow we're using
!git -c advice.detachedHead=false -C tensorflow checkout 17ce384df70

## 텐서보드 불러오기

훈련이 진행되면서 변화하는 모델의 정확도와 손실값을 그래프로 확인하기 위해 텐서보드를 불러오도록 하자.  

In [ ]:
# Delete any old logs from previous runs
!rm -rf /content/retrain_logs
# Load TensorBoard
%load_ext tensorboard
%tensorboard --logdir /content/retrain_logs

## 훈련 시작하기

다음으로, 다음의 스크립트를 실행하여 훈련을 시작하자. 스크립트는 우선, 훈련 데이터를 다운로드 할 것이다:

In [ ]:
!python tensorflow/tensorflow/examples/speech_commands/train.py \
--model_architecture=tiny_conv --window_stride=20 --preprocess=micro \
--wanted_words=${WANTED_WORDS} --silence_percentage=25 --unknown_percentage=25 \
--quantize=1 --verbosity=WARN --how_many_training_steps=${TRAINING_STEPS} \
--learning_rate=${LEARNING_RATE} --summaries_dir=/content/retrain_logs \
--data_dir=/content/speech_dataset --train_dir=/content/speech_commands_train \


## 그래프 고정하기 

훈련이 종료되면 다음 셀을 실행시켜 그래프를 고정시키도록 하자. 

In [ ]:
!python tensorflow/tensorflow/examples/speech_commands/freeze.py \
--model_architecture=tiny_conv --window_stride=20 --preprocess=micro \
--wanted_words=${WANTED_WORDS} --quantize=1 --output_file=/content/tiny_conv.pb \
--start_checkpoint=/content/speech_commands_train/tiny_conv.ckpt-${TOTAL_STEPS}

## 모델 변환

다음 셀을 실행시켜서 고정된 그래프를 TensorFlow Lite 모델 컨버터를 활용하여 변환시키도록 하자. 변환된 모델은 임베디드 기기에서 사용하기 위해 소숫점이 없이 완전히 양자화 되었다.

In [ ]:
!toco \
--graph_def_file=/content/tiny_conv.pb --output_file=/content/tiny_conv.tflite \
--input_shapes=1,49,40,1 --input_arrays=Reshape_2 --output_arrays='labels_softmax' \
--inference_type=QUANTIZED_UINT8 --mean_values=0 --std_dev_values=9.8077

다음 셀은 위의 셀에서 변환된 모델의 크기를 출력해준다. 확인하면 20KB 미만임을 알 수 있다. 

In [ ]:
import os
model_size = os.path.getsize("/content/tiny_conv.tflite")
print("Model is %d bytes" % model_size)

마지막으로 xxd를 사용하여 모델을 한 번 더 변환하자. 변환된 파일은 C++프로젝트에 포함될 수 있으며 마이크로 컨트롤러에서 사용할 수 있다. 

In [ ]:
# Install xxd if it is not available
!apt-get -qq install xxd
# Save the file as a C source file
!xxd -i /content/tiny_conv.tflite > /content/tiny_conv.cc
# Print the source file
!cat /content/tiny_conv.cc